In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# 라이브러리 버젼 체크 : 충돌방지

In [2]:
import numpy as np
np.__version__

'2.0.2'

In [3]:
import sklearn
sklearn.__version__

'1.7.1'

In [4]:
import pandas as pd
pd.__version__

'2.3.0'

# 데이터 체크

In [5]:
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition_변환.csv')
df

,나이,회사와의마찰,출장,일당,부서,거리,학력,전공,사번,만족도,...,성인여부,야근,업무평가,주변평가,근무기준시간,스톡옵션레벨,경력,전년도교육출장횟수,워라밸,현회사근속년수
0,41,Yes,1~29회,1102,영업,1,2,사회과학계열,1,2,...,Y,Yes,보통,1,80,0,8,0,1,6
1,49,No,30회 이상,279,R&D,8,1,사회과학계열,2,3,...,Y,No,좋다,4,80,1,10,3,3,10
2,37,Yes,1~29회,1373,R&D,2,2,기타,4,4,...,Y,Yes,보통,2,80,0,7,3,3,0
3,33,No,30회 이상,1392,R&D,3,4,사회과학계열,5,4,...,Y,Yes,보통,3,80,0,8,3,3,8
4,27,No,1~29회,591,R&D,2,1,자연과학계열,7,1,...,Y,No,보통,4,80,1,6,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,30회 이상,884,R&D,23,2,자연과학계열,2061,3,...,Y,No,보통,3,80,1,17,3,3,5
1466,39,No,1~29회,613,R&D,6,1,자연과학계열,2062,4,...,Y,No,보통,1,80,1,9,5,3,7
1467,27,No,1~29회,155,R&D,4,3,사회과학계열,2064,2,...,Y,Yes,좋다,2,80,1,6,0,3,6
1468,49,No,30회 이상,1023,영업,2,3,자연과학계열,2065,4,...,Y,No,보통,4,80,0,17,3,2,9


In [6]:
df[df.isna().any(axis=1)] # 결측치 0건

,나이,회사와의마찰,출장,일당,부서,거리,학력,전공,사번,만족도,...,성인여부,야근,업무평가,주변평가,근무기준시간,스톡옵션레벨,경력,전년도교육출장횟수,워라밸,현회사근속년수


In [7]:
df.columns

Index(['나이', '회사와의마찰', '출장', '일당', '부서', '거리', '학력', '전공', '사번', '만족도', '성별',
       '시급', '참여프로젝트', '근속연차', '직급', '직업만족도', '결혼여부', '월급', '이직회수', '성인여부',
       '야근', '업무평가', '주변평가', '근무기준시간', '스톡옵션레벨', '경력', '전년도교육출장횟수', '워라밸',
       '현회사근속년수'],
      dtype='object')

In [8]:
x_col = ['나이','출장','부서','학력','전공','참여프로젝트','직급','경력','전년도교육출장횟수','현회사근속년수']
y_col = ['업무평가']

# 데이터 전처리

In [9]:
# 데이터 우선 재체크
df_data = df[x_col + y_col]
X = df_data[x_col]
y = df_data[y_col]
X.shape, y.shape

((1470, 10), (1470, 1))

## 원핫인코딩 : 부서, 전공

In [10]:
from sklearn.preprocessing import OneHotEncoder

department_ohe = OneHotEncoder(sparse_output=False)
major_ohe=OneHotEncoder(sparse_output=False)

department = department_ohe.fit_transform(X[['부서']])
major = major_ohe.fit_transform(X[['전공']])

# Ordinal Encoding : 출장횟수 분류, 직급

In [17]:
print(X['출장'].unique()) # [1,2,0]
print(X['직급'].unique()) # {"영업직" : 0, "연구직" : 0, "인사사원" : 0, "엔지니어":0, "제조책임자":1,"연구직관리자":1, "인사관리자":1,"영업담당자":1,"연구관리자":2}

['1~29회' '30회 이상' '0회']
['영업직' '연구직' '엔지니어' '제조 책임자' '연구직 관리자' '인사관리자' '영업 담당자' '연구 관리자' '인사사원']


In [29]:
def ordinalEncoding(row) : 
    row = row.copy()
    manager_dict = {"영업직" : 0, "연구직" : 0, "인사사원" : 0, "엔지니어":0, "제조 책임자":1,"연구직 관리자":1, "인사관리자":1,"영업 담당자":1,"연구 관리자":2}
    if row['출장'] == "1~29회" :
        row['출장odm'] = 1
    elif row['출장'] == "30회 이상" :
        row['출장odm'] = 2
    else :
        row['출장odm'] = 0
    row['직급odm'] = manager_dict.get(row['직급'])
    return row

ordinalEncoding(X.iloc[0]) # ok

나이               41
출장            1~29회
부서               영업
학력                2
전공           사회과학계열
참여프로젝트            3
직급              영업직
경력                8
전년도교육출장횟수         0
현회사근속년수           6
출장odm             1
직급odm             0
Name: 0, dtype: object

In [30]:
X_odm = X.apply(ordinalEncoding, axis=1,)
X_odm

,나이,출장,부서,학력,전공,참여프로젝트,직급,경력,전년도교육출장횟수,현회사근속년수,출장odm,직급odm
0,41,1~29회,영업,2,사회과학계열,3,영업직,8,0,6,1,0
1,49,30회 이상,R&D,1,사회과학계열,2,연구직,10,3,10,2,0
2,37,1~29회,R&D,2,기타,2,엔지니어,7,3,0,1,0
3,33,30회 이상,R&D,4,사회과학계열,3,연구직,8,3,8,2,0
4,27,1~29회,R&D,1,자연과학계열,3,엔지니어,6,3,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,30회 이상,R&D,2,자연과학계열,4,엔지니어,17,3,5,2,0
1466,39,1~29회,R&D,1,자연과학계열,2,연구직 관리자,9,5,7,1,1
1467,27,1~29회,R&D,3,사회과학계열,4,제조 책임자,6,0,6,1,1
1468,49,30회 이상,영업,3,자연과학계열,2,영업직,17,3,9,2,0


In [31]:
X_odm[X_odm.isna().any(axis=1)] # 결측치 0건 : 정상작동

,나이,출장,부서,학력,전공,참여프로젝트,직급,경력,전년도교육출장횟수,현회사근속년수,출장odm,직급odm


In [34]:
import numpy as np
x_recol = ['나이','학력','참여프로젝트','경력','전년도교육출장횟수','현회사근속년수','출장odm','직급odm']
X_recol = X_odm[x_recol].values
X_value = np.hstack([X_recol,department,major])
X_value.shape, y.shape

((1470, 17), (1470, 1))

In [35]:
# 학습셋, 시험셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_value,y,test_size=0.2,stratify=y)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1176, 17), (294, 17), (1176, 1), (294, 1))

# sample 모델 : RandomForeseClassifier
- RandomForestClassifier로 우선 sampling 후 feature_importance_ 점검
- 이후 class_weight = "balanced" 점검
- 결과에 따라 타 모델 점검 및 smote 여부 점검

In [36]:
np.array(X_train)

array([[58.,  4.,  3., ...,  0.,  0.,  0.],
       [26.,  4.,  1., ...,  0.,  0.,  1.],
       [38.,  3.,  2., ...,  0.,  0.,  0.],
       ...,
       [31.,  4.,  3., ...,  0.,  0.,  0.],
       [34.,  2.,  3., ...,  0.,  0.,  1.],
       [22.,  2.,  4., ...,  0.,  0.,  1.]])

In [37]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
rf_model = RandomForestClassifier()
history = rf_model.fit(np.array(X_train), np.array(Y_train).ravel())

In [38]:
history

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


# 모델 결과

In [39]:
pred = rf_model.predict(np.array(X_test))
value = np.array(Y_test).ravel()

In [40]:
pred.shape, value.shape

((294,), (294,))

In [41]:
pd.crosstab(value,pred, rownames=['실제값'],colnames=['예측값'])

예측값,보통
실제값,
보통,249
좋다,45


# sample 모델 : RandomForeseClassifier : class_weight = balanced
- RandomForestClassifier로 우선 sampling 후 feature_importance_ 점검
- 이후 class_weight = "balanced" 점검
- 결과에 따라 타 모델 점검 및 smote 여부 점검

In [42]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rf_cl_balanced_model = RandomForestClassifier(class_weight="balanced")
history = rf_cl_balanced_model.fit(np.array(X_train), np.array(Y_train).ravel())

In [43]:
history

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [44]:
pred = rf_cl_balanced_model.predict(np.array(X_test))
Y_test, pred

(     업무평가
 319    보통
 228    보통
 590    보통
 335    좋다
 1020   보통
 ...   ...
 954    보통
 234    보통
 704    보통
 1372   좋다
 282    보통
 
 [294 rows x 1 columns],
 array(['보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통', '보통',
        '보통', '보통', '보통', '

In [45]:
pd.crosstab(np.array(Y_test).ravel(),pred,rownames=['실제값'],colnames=['예측값'])

예측값,보통,좋다
실제값,,
보통,248,1
좋다,45,0


In [46]:
Y_test[Y_test['업무평가']=='좋다'].value_counts() # 좋다 45건

업무평가
좋다      45
Name: count, dtype: int64

In [47]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(Y_test, pred)

In [48]:
acc # 84.4% 정확도

0.8435374149659864

In [49]:
from sklearn.metrics import classification_report
report = classification_report(Y_test, pred)
print(report)

              precision    recall  f1-score   support

          보통       0.85      1.00      0.92       249
          좋다       0.00      0.00      0.00        45

    accuracy                           0.84       294
   macro avg       0.42      0.50      0.46       294
weighted avg       0.72      0.84      0.78       294



# OneHotEncoding + OrdinalEncoding 결과 정확도가 84.4%로 약 0.3% 가량 감소한 형태
- OneHotEncoding이 필요가 없을 수도 있음
- OrdinalEncoding 또한 필요 없을 수 있음

## 부서/전공은 OneHotEncoding(점수 평가에 서열을 갖는 데이터가 아니기 때문)
## 출장횟수, 직급은 Ordinal Encoding(서열을 갖는 인코딩 방식)을 활용해보기

```
1. 왜 라벨 인코딩이 더 잘 나왔나?
트리 기반 모델 특성
RandomForest, XGBoost, LightGBM 같은 트리 모델은 수치 크기를 순서·서열로 해석하지 않고, 단순히 분할 기준으로만 사용
그래서 라벨 인코딩으로 숫자만 부여해도, 그 숫자가 ‘크다/작다’라는 의미보다는 그룹을 나누는 기준으로 활용

OneHotEncoding의 희석 효과
출장횟수(없음/보통/많음), 직급(9단계)처럼 명확한 서열 관계나 가중치가 있는 변수는 OneHot으로 변환하면
→ 정보가 분리되어 서열·강도 관계가 사라지고, 모델이 이를 별도로 학습해야 해서 성능이 저하 가능성

특정 Feature의 영향력
예를 들어 직급은 분명히 ‘높음 → 업무평가 영향력 다름’ 같은 구조가 있을 수 있는데, OneHot은 이런 정보를 누락시키는 방향
```